# Verifiably Safe AI via Logically Linked Envelopes (VerSAILLE)
Now that we have proven our model in KeYmaera (see [02-KeYmaera.ipynb](./02-KeYmaera.ipynb)) it is time to think about how we can use the obtained results to verify the safety of a neural network.

To this end, we make use of the following observation:  
> If all the behavior of our neural network is also behavior admitted by our control envelope, then the safety guarantee of the control envelope carries over to the neural network.

To deduce the behavior allowed by the control envelope, we begin by synthesizing a monitoring formula using ModelPlex.
The monitor will look something like this, note that it is a disjunction at the top level:
```
x1post >= x2post &
x1post >= x3post &
prel + T() * vrel + (-A()) * T()^2 / 2 >= (vrel + T() * (-A()))^2 / (2 * B()) &
(
  -2 * vrel / (-A()) > T() |
  prel + (-2 * vrel / (-A())) * vrel + (-2 * vrel / (-A()))^2 * (-A()) / 2 >=
  (vrel + (-2 * vrel / (-A())) * (-A()))^2 / (2 * B())
) &
0 <= T() & arelpost = -A() & prelpost = prel & tpost = 0 & vrelpost = vrel |
x2post >= x1post &
x2post >= x3post &
prel + T() * vrel >= vrel^2 / (2 * B()) &
0 <= T() & arelpost = 0 & prelpost = prel & tpost = 0 & vrelpost = vrel |
x3post >= x2post &
x3post >= x1post &
prel + T() * vrel + B() * T()^2 / 2 >= (vrel + T() * B())^2 / (2 * B()) &
(
  -2 * vrel / B() > T() |
  prel + (-2 * vrel / B()) * vrel + (-2 * vrel / B())^2 * B() / 2 >=
  (vrel + (-2 * vrel / B()) * B())^2 / (2 * B())
) &
0 <= T() & arelpost = B() & prelpost = prel & tpost = 0 & vrelpost = vrel
```
Note, that for (potentially) every variable the formula contains a pre value (e.g. `vrel`) and a post value (e.g. `vrelpost`).
Importantly, only some of the variables are determined by the neural network.
In particular, since the neural network does not determine `T()`,`arelpost`,`prelpost`,`tpost`,`vrelpost` etc. we can remove these equality constraints from all conjunctions.
Doing some more cleanup of unnecessary brackets (we do not require the brackets after constants like `B()`),
this yields:
```
x1post >= x2post &
x1post >= x3post &
prel + T * vrel + (-A) * T^2 / 2 >= (vrel + T * (-A))^2 / (2 * B) &
(
  -2 * vrel / (-A) > T |
  prel + (-2 * vrel / (-A)) * vrel + (-2 * vrel / (-A))^2 * (-A) / 2 >=
  (vrel + (-2 * vrel / (-A)) * (-A))^2 / (2 * B)
) |
x2post >= x1post &
x2post >= x3post &
prel + T * vrel >= vrel^2 / (2 * B) |
x3post >= x2post &
x3post >= x1post &
prel + T * vrel + B * T^2 / 2 >= (vrel + T * B)^2 / (2 * B) &
(
  -2 * vrel / B > T |
  prel + (-2 * vrel / B) * vrel + (-2 * vrel / B)^2 * B / 2 >=
  (vrel + (-2 * vrel / B) * B)^2 / (2 * B)
)
```
We could now simply analyze whether this formula is satisfied by all input-output behavior of our neural network.
However, before we do so we can further restrict the analyzed input space.
Note, that we only care about behavior that is actually reachable, thus we can constrain the input space via:
- Input Variable Bounds (e.g. `0<=prel & prel <=100`)
- Invariants we know about our system
Since our tool uses linear approximations of nonlinear formulas, we actually **require** interval bounds for all variables. However, they may be arbitrarily large.
This yields the following formula that we want to check on our NN:
```
(
(0<=prel & prel<=100 & -34 <= vrel & vrel <= 34 &
-100 <= x1post & x1post <= 100  & -100 <= x2post & x2post <= 100  & -100 <= x3post & x3post <= 100) &
(prel >= 0 & prel >= vrel^2 / (2*B))
)
->
(
x1post >= x2post &
x1post >= x3post &
prel + T * vrel + (-A) * T^2 / 2 >= (vrel + T * (-A))^2 / (2 * B) &
(
  -2 * vrel / (-A) > T |
  prel + (-2 * vrel / (-A)) * vrel + (-2 * vrel / (-A))^2 * (-A) / 2 >=
  (vrel + (-2 * vrel / (-A)) * (-A))^2 / (2 * B)
) |
x2post >= x1post &
x2post >= x3post &
prel + T * vrel >= vrel^2 / (2 * B) |
x3post >= x2post &
x3post >= x1post &
prel + T * vrel + B * T^2 / 2 >= (vrel + T * B)^2 / (2 * B) &
(
  -2 * vrel / B > T |
  prel + (-2 * vrel / B) * vrel + (-2 * vrel / B)^2 * B / 2 >=
  (vrel + (-2 * vrel / B) * B)^2 / (2 * B)
)
)
```